In [1]:
# import libraries

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw
from skimage import io
import time
import tifffile as tif

In [2]:
# CHANGE WITH EVERY NEW DATASET
STACKSIZE = 1000
DATA = "D:/Hanna Analysis/Orientation_13_7_fused/13_7_fused_binary_cropped.tif"
COORFILE = np.asarray(pd.read_excel(r"D:/Hanna Analysis/Orientation_13_7_fused/LayerCoor_13_7_fused.xlsx")) # , sheet-name = "V1"
# number of x,y coordinate tuples in each line
LINE_COORCOUNT = 22
# total number of layers specified in coordinate file
LAYER_COUNT = 1

In [3]:
# Fixed variables
# set the filling of the polygon, can be anything but 0, does not matter, because that is the part we do not need
EMPTY = 2
BLACK = 0
# number of x,y coordinate tuples in each polygon
COORCOUNT = 2*LINE_COORCOUNT

In [4]:
def straightening_coor(coorfile, layernumber, filename):
    upper_layertop = []
    upper_layerbot = []
    lower_layertop = []
    lower_layerbot = []

    coorfile = coorfile[:,1:]

    for n in range(LINE_COORCOUNT):
        upper_layertop.append([coorfile[layernumber-1][n],coorfile[layernumber-1][LINE_COORCOUNT+n]])
        upper_layerbot.append([coorfile[layernumber+(LAYER_COUNT)][n],coorfile[layernumber+(LAYER_COUNT)][LINE_COORCOUNT+n]])
        lower_layertop.append([coorfile[layernumber][n],coorfile[layernumber][LINE_COORCOUNT+n]])
        lower_layerbot.append([coorfile[layernumber+(LAYER_COUNT+1)][n],coorfile[layernumber+(LAYER_COUNT+1)][LINE_COORCOUNT+n]])

    straight_top = []
    straight_bot = []

    for i in range(LINE_COORCOUNT):
        dis_y_top = (lower_layertop[i][1]-upper_layertop[i][1])/2
        dis_y_bot = (lower_layerbot[i][1]-upper_layerbot[i][1])/2
        y_top = dis_y_top + upper_layertop[i][1]
        y_bot = dis_y_bot + upper_layerbot[i][1]
        upper_layertop[i][1] = y_top
        upper_layerbot[i][1] = y_bot
        straight_top = upper_layertop
        straight_bot = upper_layerbot


    itrpltd_dist = []

    # calculate the difference between top and bottom polygon y-coordinates, the difference
    # by stack size -1 = number of images that need to be interpolated

    for n in range(LINE_COORCOUNT):
         diff = straight_top[n][1]-straight_bot[n][1]
         dist = diff/(STACKSIZE-1)
         itrpltd_dist.append(dist)

    # fill a stack of size stack_size (size of data stack) with coordinates of top layer -> will be changed for each image in next step
    arr1 = np.array(straight_top)
    stack_coordinates = [arr1]*STACKSIZE
    stack_coord = np.asarray(stack_coordinates)

    # get an array for the interpolated coordinates for all images -> image coordinate - interpolated distance

    for n in range(STACKSIZE):        
        for i in range(LINE_COORCOUNT):         
            stack_coord[n][i][1] = stack_coord[n][i][1] - itrpltd_dist[i]*n
    
    straight_coord = []
    straight_coordinates = []
    straight_coordinates.append([100]*24)

    for n in range(STACKSIZE):
        straight_coord = []
        for i in range(LINE_COORCOUNT):
                straight_coord.append(stack_coord[n][i][0])
                straight_coord.append(stack_coord[n][i][1])      
        straight_coordinates.append(straight_coord)

    straight_coordinates = np.asarray(straight_coordinates)
    straight_coord_round = np.around(straight_coordinates,2) 

    # save to excel file with filename  
    np.savetxt(filename, straight_coord_round, delimiter=',') 

In [5]:
straight_layer1 = straightening_coor(COORFILE, 1, "V1 Layer1 StraightCoor.csv")
# straight_layer2 = straightening_coor(COORFILE, 2, "V1 Layer2 StraightCoor.csv")
# straight_layer3 = straightening_coor(COORFILE, 3, "V1 Layer3 StraightCoor.csv")

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (1001,) + inhomogeneous part.